# unet

> Fill in a module description here

In [ ]:
#| default_exp unet

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn

In [ ]:
#| exports
class BasicUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        self.down_layers = nn.ModuleList([
            nn.Conv2d(
                in_channels=in_channels, out_channels=32,
                kernel_size=5, padding=2
            ),
            nn.Conv2d(
                in_channels=32, out_channels=64,
                kernel_size=5, padding=2
            ),
            nn.Conv2d(
                in_channels=64, out_channels=64,
                kernel_size=5, padding=2
            ),
            
        ])
        
        self.up_layers = nn.ModuleList([
            nn.Conv2d(
                in_channels=64, out_channels=64,
                kernel_size=5, padding=2
            ),
            nn.Conv2d(
                in_channels=64, out_channels=32,
                kernel_size=5, padding=2
            ),
            nn.Conv2d(
                in_channels=32, out_channels=out_channels,
                kernel_size=5, padding=2
            ),
        ])
        self.act = nn.SiLU()
        self.downscale = nn.MaxPool2d(kernel_size=2)
        self.upscale = nn.Upsample(scale_factor=2)
    
    def forward(self, x):
        h = []
        for i, layer in enumerate(self.down_layers):
            x = self.act(layer(x))
            if i < 2:
                h.append(x)
                x = self.downscale(x)
        
        for i, layer in enumerate(self.up_layers):
            if i > 0:
                x = self.upscale(x)
                x += h.pop()
            x = self.act(layer(x))
        
        return x

In [ ]:
net = BasicUNet()
x = torch.rand(8, 1, 28, 28)
net(x).shape

torch.Size([8, 1, 28, 28])